In [1]:
from BackgammonModel import *

In [2]:
model = BackgammonModel( \
    input_size = 198,
    hidden_size = 50,
    output_size = 1,
    trace_decay = 0.7,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/50/checkpoint_358000.pt',
    eval_interval = 100
)

Loading saved model: ./checkpoints/50/checkpoint_358000.pt


In [3]:
model2 = BackgammonModel( \
    input_size = 198,
    hidden_size = 100,
    output_size = 1,
    trace_decay = 0.7,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/100/checkpoint_75000.pt',
    eval_interval = 1000
)

Loading saved model: ./checkpoints/100/checkpoint_75000.pt


In [4]:
env = Env()
model.evaluate(env, [RandomAgent(player = 0), 
                     RandomAgent(player = 1)])

100%|██████████| 100/100 [00:07<00:00, 14.10it/s]


{0: 49, 1: 51, <Player.EMPTY: 2>: 0}

In [5]:
model.evaluate(env, [TDAgent(player = 0, net = model2, env = env),
                     RandomAgent(player = 1),
                     ])

100%|██████████| 100/100 [00:13<00:00,  7.48it/s]


{0: 94, 1: 5, <Player.EMPTY: 2>: 1}

In [6]:
env = Env()
model.evaluate(env, [RandomAgent(player = 0),
                     TDAgent(player = 1, net = model2, env = env)])

100%|██████████| 100/100 [00:28<00:00,  3.48it/s]


{0: 54, 1: 6, <Player.EMPTY: 2>: 40}

In [ ]:
env = Env()
model.evaluate(env, [TDAgent(player = 0, net = model2, env = env),
                     TDAgent(player = 1, net = model, env = env)])

 48%|████▊     | 48/100 [00:09<00:11,  4.68it/s]

In [ ]:
env = Env()
model.evaluate(env, [TDAgent(player = 0, net = model, env = env),
                     TDAgent(player = 1, net = model2, env = env)])

In [ ]:
%load_ext line_profiler
model = BackgammonModel( \
    input_size = 198,
    hidden_size = 50,
    output_size = 1,
    trace_decay = 0.7,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/50/checkpoint_358000.pt',
    eval_interval = 100
)
env = Env()
agents = [RandomAgent(player = 0), TDAgent(player = 1, net = model, env = env)]

In [ ]:
p, _, _, _ = env.reset()
agent = agents[p]
legal_actions = env.get_legal_actions()
print(legal_actions)
game = env.game

In [ ]:
%lprun -f game.observation_tensor game.observation_tensor(0)

In [ ]:
%lprun -f game.step game.step(legal_actions[0])

In [ ]:
%lprun -f agent.choose_best_action agent.choose_best_action(legal_actions)

In [ ]:
%lprun -f model.evaluate model.evaluate(env, agents, num_episodes = 1)


In [ ]:
from Backgammon import *
game = Backgammon(verbose=False, max_turns = 200)


In [ ]:
%lprun -f game.get_legal_actions game.get_legal_actions()

In [ ]:
%lprun -f game.legal_checker_moves_outer game.legal_checker_moves_outer(0, game.dice)

In [ ]:
%lprun -f game.legal_checker_moves game.legal_checker_moves(game.cur_player, game.dice)

In [ ]:
a = game.legal_checker_moves(game.cur_player, game.dice)[0]
a

In [ ]:
%lprun -f game.apply_move game.apply_move(a)

In [ ]:
%lprun -f game.update_board_pos game.update_board_pos(15, 1, 1)

In [ ]:
env = Env()
env.reset()
legal_actions = env.get_legal_actions()
print(legal_actions)

In [ ]:
def choose_best_action(actions):
    obs_ = []
    tots = []
    for a in actions:
        _, _, obs, _, _ = env.step(a)
        _ = env.step_back(a)
        tot = 0
        for i in range(len(obs)):
            tot +=  obs[i]*i
        tots.append(tot)
    return tots
tots_no_pool = choose_best_action(legal_actions)
print(tots_no_pool)

In [ ]:
from multiprocessing import Pool

def g(a):
    _, _, obs, _, _ = env.step(a)
    _ = env.step_back(a)
    tot = 0
    for i in range(len(obs)):
        tot +=  obs[i]*i
    return tot
def f(x):
    return x*x

with Pool(5) as p:
    tots_pool = p.map(g, legal_actions)
print(tots_pool)

In [ ]:
%%timeit
_ = choose_best_action(legal_actions)


In [ ]:
%%timeit
with Pool(5) as p:
    _ = p.map(g, legal_actions)

In [ ]:
p = Pool(4)

In [ ]:
%%timeit
_ = p.map(g, legal_actions)